# Template - Prefix Sums

Prefix sums are a technique for obtaining cumulative sums of arrays. Given an array `A`, the prefix sum array `P` may store the following at `P[i]`:
- Cumulative sum in `A` from `A[0]` to `A[i]`
- Cumulative sum in `A` of all values left of `i`; `P[0] = 0` and `P[i] = A[0] + A[1] + ... + A[i-1]`. 
- Any of the above, but from the right, e.g. the sum from `A[i]` to `A[-1]`
- Any of the above, but `P` stores a cumulative product or some other operation.

Prefix sums are often useful in situations asking for a count of particular types of subarrays. Problems solvable with prefix sums can frequently also be solved with dynamic programming or sliding windows. 

A useful template for leveraging prefix sums is:
```
1. Generate required prefix sum array/s
2. Traverse arrays and count instances of complements wrt the requested value

```

## Example: [Subarray sum equals K](https://leetcode.com/problems/subarray-sum-equals-k/), [Binary subarrays with sum](https://leetcode.com/problems/binary-subarrays-with-sum/)

These two problems are identical; using binary subarrays causes a subset of possible cases. To solve them, we can imagine a prefix sum array with `sum of values left of i`:

```
arr  = [4,-1, 1, 2, 3]
pref = [0, 4, 3, 4, 6, 7]
```
To solve this, we need to find all indexes `i,j` with `i<j` where `arr[j] - arr[i] == k` or `arr[j] == arr[i] + k`. Each `pref[n]` can be a value of `j` if we've seen `pref[n]-k` previously in `pref`, and it can otherwise be a possible value for `i`. In the solution below, we compress some operations and don't keep an explicit prefixes array (rather a default dict). 

In [10]:
from typing import List
from collections import defaultdict

class Solution:
    def subarraySum(self, nums: List[int], k: int) -> int:
        subarrays = csum = 0
        prefixes = defaultdict(int)
        prefixes[csum+k] += 1
        for val in nums:
            csum += val 
            subarrays = subarrays + prefixes[csum]           
            prefixes[csum+k] += 1
        return subarrays

s = Solution()
cases = [
    ([1,1,1], 2, 2),
    ([1,2,3], 3, 2),
    ([4,-1,1,2,3], 3, 3)
]
for arr, val, expected in cases:
    actual = s.subarraySum(arr, val)
    assert expected == actual, f"{arr,val}: {expected} != {actual}"

## Example: [Product of Array Except Self](https://leetcode.com/problems/product-of-array-except-self/)

This `O(n)` solution uses left and right prefix products and no division. `solution[i] =  left_prefix[i-1] * right_prefix[i+1]`.

In [ ]:
from collections import deque
from typing import List

class Solution:
    def productExceptSelf(self, nums: List[int]) -> List[int]:
        r_prefix = deque([])
        l_prefix = deque([])
        l_prod = r_prod = 1
        for i, val in enumerate(nums):
            r_prod *= val
            r_prefix.append(r_prod)
            l_prod *= nums[len(nums)-1-i]
            l_prefix.appendleft(l_prod)
        
        solution = []
        for i, _ in enumerate(nums):
            r = r_prefix[i-1] if i > 0 else 1
            l = l_prefix[i+1] if i < len(nums)-1 else 1
            solution.append(r*l)
        return solution

s = Solution()
assert s.productExceptSelf([1,2,3,4]) == [24, 12, 8, 6]
assert s.productExceptSelf([0,1,2,4]) == [8, 0, 0, 0]
assert s.productExceptSelf([0,1,2,0]) == [0, 0, 0, 0]
assert s.productExceptSelf([1,2]) == [2,1]


## Example: [Two sum](https://leetcode.com/problems/two-sum/)

We don't need prefix sums here, but we can leverage `complementary elements` by computing a list of complements with regard to the target and then scanning it for the correct complement also in the list.
```
tgt = 9
A: [2, 7, 11, 15]
C: [7, 2, -2, -6] # C[i] = 9 - A[i]
    ^  ^
```
However, we can compress this down to one pass storing the required complement for each `A[i]`; if `9 = a,b` for `a,b` in `C`, then by the time we find `b`, we will already know `a`:

In [ ]:
class Solution:
    def twoSum(self, nums: List[int], target: int) -> List[int]:
        comps = {}
        for i, val in enumerate(nums):
            if val in comps and i != comps[val]:
                return [i, comps[val]]
            comps[target - val] = i

s = Solution()
cases = [
    ([2, 7, 11, 15], 9, [0,1]),
    ([-1,-2,-3,-4,-5], -8, [2,4])
]
for arr, target, expected in cases:
    actual = s.twoSum(arr, target)
    assert sorted(expected) == sorted(actual), f"arr,target: {expected} != {actual}"
